In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
def sampling(df):
    label_counts = df["Set_Fingerprint"].apply(lambda labels: frozenset(labels)).value_counts()
    label_combinations_to_remove = label_counts[label_counts == 1].index
    df = df[~df['Set_Fingerprint'].apply(lambda x: frozenset(x)).isin(label_combinations_to_remove)]

    label_counts = df["Set_Fingerprint"].apply(lambda labels: frozenset(labels)).value_counts()

    label_counts_to_sample = label_counts[label_counts >= 2]
    selected_rows = pd.DataFrame()
    for label in label_counts_to_sample.index:
        rows_with_labels = df[df["Set_Fingerprint"].apply(lambda labels: frozenset(labels)).isin([label])]
        num_rows_to_select = int(len(rows_with_labels) * 0.1)
        if num_rows_to_select == 0 or num_rows_to_select == 1:
            num_rows_to_select = 2
        selected_rows = pd.concat([selected_rows, rows_with_labels.sample(n=num_rows_to_select)])
    return selected_rows

In [3]:
def split_train_test(X, y, train_size, random_state):
    y_str = y.apply(lambda x: ', '.join(sorted(x)))
    X_train, X_test, y_train_str, y_test_str = train_test_split(X, y_str, train_size=train_size, stratify=y_str, random_state=random_state)
    y_train = y_train_str.apply(lambda x: x.split(', '))
    y_test = y_test_str.apply(lambda x: x.split(', '))

    return X_train, X_test, y_train, y_test